In [ ]:
import matplotlib.pyplot as plt
import json
import pandas as pd
import numpy as np

with open('data/general/matches_2017_2018_v1.json') as matches_json:
    matches = pd.read_json(matches_json)
    

In [ ]:
for id in matches['id']:
    with open('data/match_' + str(id) + '/match_data_' + str(id) + '.json', 'r') as file:
        teams = json.load(file)
        
    refreee_id = ''
    home_team_id = teams[1]['teamId']
    away_team_id = teams[2]['teamId']
    home_team_player = []
    away_team_player = []
    
    team_dict = {}
    team_dict[0] = 0
    team_dict[home_team_id] = 1
    team_dict[away_team_id] = 2
    
    with open('data/match_' + str(id) + '/players_data_' + str(id) + '.json', 'r' ) as file:
        player = json.load(file)
        
    for p in player:
        if p['teamId'] == home_team_id:
            home_team_player.append(str(p['teamId']) + str(p['jerseyNumber']) )
        elif p['teamId'] == away_team_id:
            away_team_player.append(str(p['teamId']) + str(p['jerseyNumber']))
        else:
            referee_id = str(p['teamId']) + str(p['jerseyNumber'])
            
    home_team_player = sorted(home_team_player)
    home_val = np.arange(1, len(home_team_player) + 1)
    home_dict= dict(zip(home_team_player, home_val))
    
    away_team_player = sorted(away_team_player)
    away_val = np.arange(15, len(away_team_player)+15)
    away_dict = dict(zip(away_team_player, away_val))
    
    player_dict_only_team_members = {**home_dict, **away_dict}
    player_dict = player_dict_only_team_members.copy()
    player_dict[referee_id] = 0
    
    data_height = 6000
    data_width = 87
    label_team_width = 3
    label_player_width = 29
    
    x_data = np.zeros(shape=(data_height, data_width), dtype=np.float)
    y_player_data = np.zeros(shape=(data_height, label_player_width), dtype=np.int)
    y_team_data = np.zeros(shape=(data_height, label_team_width), dtype=np.int)
    x_position = np.zeros(shape=(data_height, 29, 2), dtype=np.float)
    
    with open('data/match_' + str(id) + '/per_sec_data_' + str(id) + '.json', 'r') as file:
        data_persec = json.load(file)
        
    
    
    for d in data_persec:
        i = int(d['minute'])*60 + int(d['second'])
        j = player_dict[str(d['teamId']) + str(d['jerseyNumber'])]
        k = team_dict[d['teamId']]
        
        x_position[i][j][0] = d['xpos']
        x_position[i][j][1] = d['ypos']
        
        x_data[i][j*3] = round(d['xpos'], 2)
        x_data[i][j*3+1] = round(d['ypos'],2)
        x_data[i][j*3+2] = round(d['speed'],2)
        
        #print ('j - ' + x_data[i][j*3] '/' +  x_data[i][j*3+1] + '/' + x_data[i][j*3+2] )
        
        if d['teamId'] == d['hasballTeamId']:
            y_team_data[i][k] = 1
            if d['jerseyNumber'] == d['hasballJerseyNumber']:
                y_player_data[i][j] = 1
    
   
            
    with open('data/match_' + str(id) + '/x_position.npy', 'wb') as file:
        np.save(file, x_position)
    
    with open('data/match_' + str(id) + '/x_data.npy', 'wb') as file:
        np.save(file, x_data)
    
    with open('data/match_' + str(id) + '/y_team_data.npy', 'wb') as file:
        np.save(file, y_team_data)
    
    with open('data/match_' + str(id) + '/y_player_data.npy', 'wb') as file:
        np.save(file, y_player_data)
        
    y_pass_time = []
    y_pass_data = []
    
    with open('data/match_' + str(id) + '/pass_data_' + str(id) + '.json', 'r') as file:
        data_pass = json.load(file)
    
    player_dict = player_dict_only_team_members.copy()
    player_dict['0-1'] = 0
    
    for d in data_pass:
        i = int(d['minute'])*60 + int(d['second'])
        if d['success'] and i > 60:
            try:
                j = player_dict[str(d['toTeamId']) + str(d['toJerseyNumber'])]
                if j > 0:
                    pass_data = np.zeros(shape=(28), dtype=np.int)
                    pass_data[j-1] = 1
                    y_pass_data.append(pass_data)
                    y_pass_time.append(i)
            except:
                pass
    
    y_pass_data = np.asarray(y_pass_data)
    y_pass_time = np.asarray(y_pass_data)
                
    with open('data/match_' + str(id) + '/y_pass_data.npy', 'wb') as file:
                np.save(file, y_pass_data)
    
    with open('data/match_' + str(id) + '/y_pass_time.npy', 'wb') as file:
                np.save(file, y_pass_time)
    